In [1]:
print("hii")

hii


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os

In [3]:
%pwd

'c:\\Users\\Sanish Bhagat\\project\\derma_ai\\research'

In [4]:
img_size   = 224        # You can try 256 later
batch_size = 16         # For small dataset, 8–32 is fine
num_classes = 5         # Acne, AK, BCC, Eczema, Rosacea

train_dir = "../dataset/train"
test_dir  = "../dataset/testing"

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2,
    rotation_range=15,
    zoom_range=0.15,
    width_shift_range=0.08,
    height_shift_range=0.08,
    brightness_range=[0.7, 1.3],
    horizontal_flip=True,
    fill_mode="nearest"
)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

Found 355 images belonging to 5 classes.


In [6]:
val_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

Found 85 images belonging to 5 classes.


In [7]:
test_datagen = ImageDataGenerator(rescale=1.0/255)

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 185 images belonging to 5 classes.


In [8]:
base_model = MobileNetV2(
    input_shape=(img_size, img_size, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

In [9]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(5, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [10]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    ),
    ModelCheckpoint(
        "derma_mobilenet.keras",   # use new format instead of .h5
        monitor='val_accuracy',
        save_best_only=True
    )
]

history = model.fit(
    train_gen,
    epochs=20,
    validation_data=val_gen,
    callbacks=callbacks
)

Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 28s 847ms/step - accuracy: 0.3296 - loss: 1.6936 - val_accuracy: 0.5412 - val_loss: 1.3590
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 14s 599ms/step - accuracy: 0.5465 - loss: 1.1889 - val_accuracy: 0.4588 - val_loss: 1.4069
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 14s 619ms/step - accuracy: 0.6000 - loss: 1.0460 - val_accuracy: 0.5647 - val_loss: 1.1218
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 14s 588ms/step - accuracy: 0.6225 - loss: 0.9400 - val_accuracy: 0.4471 - val_loss: 1.2377
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 15s 652ms/step - accuracy: 0.6930 - loss: 0.8366 - val_accuracy: 0.5765 - val_loss: 1.2041
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 15s 629ms/step - accuracy: 0.6901 - loss: 0.8116 - val_accuracy: 0.5765 - val_loss: 1.1732
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 14s 624ms/step - accuracy: 0.7099 - loss: 0.7885 - val_accuracy: 0.5882 - val_loss: 1.0635
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 15s 649ms/step - accuracy: 0.7352 - loss: 0.6841 - val_accu

In [11]:
test_loss, test_acc = model.evaluate(test_gen)
print("Test accuracy:", test_acc)

12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 248ms/step - accuracy: 0.6649 - loss: 0.9395
Test accuracy: 0.6648648381233215


In [12]:
model.save("../derma_mobilenet.keras")